In [1]:
# 목표: '검증세트' 가 필요한 이유를 이해하고 교차 검증에 대해 배웁니다.
#      그리드 서치와 랜덤 처시를 이용해 최적의 성능을 내는 하이퍼 파라미터를 찾습니다

# 핵심키워드 : 검증세트, 교차검증, 그리드 서치, 랜덤 서치

# 핵심 패키지와 함수 : cross_validate(), scoring=, cv=, n_jobs=, return_train_score=
#                    GridSearchCV
#                    RandomizedSearchCV

# 데이터셋 : wine.csv

# ■교차 검증과 그리드 서치

In [2]:
# 하이퍼파라미터
# max_depth=  3 외에 다른 값들을 주어가며 '테스트세트'로 평가해보면
#   가장좋은 max_depth= 값을 찾아낼것 같다...

# 문제점! 테스트세트 에만 잘 맞는 모델이 만들어질 가능성이 커진다

# ★ 그래서 '테스트세트' 는 마지막에 딱 한번만 사용하는게 좋다!

# max_depth= 와 같은 하이퍼파라미터 튜닝은 어떻게 하면 좋을까?


# 검증세트 (validation set)


![](https://velog.velcdn.com/images/jiazzang/post/2b28c228-5b4c-41ee-82c3-4fdf48b9fda7/image.PNG)

In [3]:
# 모델이 과대적합인지 과소적합인지 판단하기 위해
# '테스트 세트' 대신! '훈련세트' 를 '또' 나누어 '검증세트'를 만듭니다


In [4]:
# 검증 데이터 세트를 사용하여 하이퍼 파라미터 튜닝한 모델을 평가한다.

# ※ 보통은 전체 데이터의 20~30% 를 테스트세트와 검증세트로 떼어놓는다. (이는 문제 사안에 다르다)


# 홀드 아웃(Hold-out) 검증

- step1 훈련세트로 → 모델을 훈련
- step2 검증세트로 모델을 평가 → 하이퍼 파라미터를 바꿔가며 최적의 하이퍼 파라미터를 가진 가장 좋은 모델을 고름
- step3 훈련세트와 검증세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련.
- step4 테스트세트 -> 모델을 최종적으로 평가. 최종점수! ←  실전에서의 성능기대 수치.

![](https://t1.daumcdn.net/cfile/tistory/994042405E24E8081C)

# 데이터 준비

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [6]:
base_path = r'/content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)'

In [7]:
file_path = os.path.join(base_path, 'wine.csv')
wine_df = pd.read_csv(file_path)

wine_df

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [8]:
# 'class' 컬럼은 target 으로 사용하도 나머지 열은 특성 배열에 저장함.

data = wine_df[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine_df['class'].to_numpy()

# 검증 세트 나누기

In [9]:
from sklearn.model_selection import train_test_split
# 우선 '훈련세트' '테스트세트' 를 나눈다   8:2
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [10]:
# 그 다음에,
# train_input 와 train_target 을 다시 train_test_split() 에 넣어
#  훈련세트 와 검증세트로 나눕니다.  검증세트는 train_input 의 약 20% 정도.

# 훈련세트: sub_input, sub_target
# 검증세트: val_input, val_target

sub_input, val_input, sub_target, val_target = \
  train_test_split(train_input, train_target, test_size=0.2, random_state=42)


In [11]:
# 확인
print(train_input.shape, sub_input.shape, val_input.shape, test_input.shape)



(5197, 3) (4157, 3) (1040, 3) (1300, 3)


In [12]:
# 이제 훈련세트와 검증세트를 사용해 모델을 만들고 평가해보자

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target)) # 훈련셋 점수
print(dt.score(val_input, val_target)) # 검증셋 점수!


0.9971133028626413
0.864423076923077


In [ ]:
# 위 모델은 overfit 되어있다
# 하이퍼파라미터를 바꾸어 가면서 '더 좋은 모델'을 찾아야 된다


# ↓ 그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있습니다.
# ↓ 그 전에 검증세트 관련해 '교차검증' 을 알아보자.

None

# 교차 검증 (Cross validation)

In [ ]:
# 문제점]
# 원래 훈련데이터를 또 쪼개었다..

# 문제1. 훈련시켜야 할 데이터 양이 줄었다
# 문제2. 훈련세트를 확보하기 위해 검증세트를 줄이면 검증점수가 들쑥날쑥해진다.

# 이 문제를 해결하기 위해 '교차검증 (cross validation)'을 사용해
#   -> 안정적인 점수도 얻고
#   -> 훈련에 더 많은 데이터 사용 가능.


**교차 검증** 은 검증세트를 떼어 내어 평가하는 과정을 여러번 '반복'합니다.
그 다음 이 '결과' 를 평균하여 최종 검증 점수를 얻습니다.

k 등분으로 떼어내어 '반복' 하여 교차검증 하는 것을 **'k-fold cross validation (k폴드 교차검증)'** 이라 한다

![](https://wikidocs.net/images/page/223699/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2023-11-30_152347.png)

In [ ]:
# 보통 5-fold, 10-fold 교차검증을 많이 사용.
# 비록, 검증세트가 줄어들었지만 각 폴드에서 계산한 검증점수를 '평균' 하기 때문에
# 안정적인 점수로 여겨도 된다

# ↓사이킷런 에는 cross_validate() 라는 교차 검증 함수를 제공해준다

## cross_validate()

In [13]:

from sklearn.model_selection import cross_validate

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

```python
sklearn.model_selection.cross_validate(
  estimator,  # 모델
  X,          # 입력데이타
  y=None,     # 타겟 데이타
  *,
  groups=None,
  scoring=None,
  cv=None,   # fold 개수, 기본은 5
  n_jobs=None,  # cpu 코어 사용 개수
  verbose=0,
  fit_params=None,
  params=None,
  pre_dispatch='2*n_jobs',
  return_train_score=False, # 검증 점수뿐만 아니라 훈련세트에 대한 점수도 같이 반환
  return_estimator=False,
  return_indices=False,
  error_score=nan)
```

In [27]:
# 검증세트 분리하기 '전'의 '훈련세트 전체'를 넘겨준다!
scores = cross_validate(dt, train_input, train_target)
scores

# 결과가 dict 다


{'fit_time': array([0.00999784, 0.00831413, 0.0087831 , 0.00854516, 0.00819254]),
 'score_time': array([0.00116181, 0.00106645, 0.00103283, 0.00102067, 0.0009954 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [ ]:
# 기본적으로 5-fold 교차검증 수행 (cv= : 폴드수)
# fit_time, score_time : 각각 모델을 '훈련'하는 시간과 '검증'하는 시간
# test_score: 모델의 각 폴드별 검증 점수


In [28]:
# 교차검증 최종점수는 5개의 검증점수의 평균
np.mean(scores['test_score'])


np.float64(0.855300214703487)

In [ ]:
# 주의!
#  cross_validate() 는 훈련세트를 섞어 폴드를 나누지 않는다.
#  ※ train_test_split() 은 전체 데이터 섞은후 훈련세트 준비했었다.

# 교차검증시 훈련세트 섞으려면 splitter(분할기) 를 지정해주어야 한다.
#     cross_validate() 는 기본적으로
#     - 회귀모델 은 'KFold 분할기' 사용
#     - 분류모델 은 StratifiedKFold 사용.


In [29]:
from sklearn.model_selection import StratifiedKFold

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold

```python
class sklearn.model_selection.StratifiedKFold(
  n_splits=5,   # fold 개수
  *,
  shuffle=False,   # 섞을지 여부
  random_state=None)
```


In [32]:
# 매개변수 기본값이 아래와 같다.
#   n_splits=5,   # fold 개수
#   shuffle=False  # 섞을지 여부

# 즉 앞서 수행한 교차 검증은 다음 코드와 동일 합니다


scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# ↓ 만약 훈련 세트를 '섞은 후' +  '10-폴드' 교차검증을 수행하려면 다음과 같이 작성한다

In [33]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
# 모델의 매개변수(하이퍼 파라미터) 를 바꿔가면서 가장 성능이 좋은 모델을 찾아보자.
# ↑ 이때! '테스트 세트'를 사용하지 않고 교차검증을 통해 좋은 모델을 고를거다


In [ ]:
# ※ 참고
# cross_validate() 외에도

# cross_val_score()  사이킷런에서 교차 검증을 조금 더 편리하게 수행할 수 있게 해준다.
#   검증할 세트 설정, 반복적으로 학습과 테스트 데이터 인덱스 추출,
#   반복적으로 학습과 예측 수행, 성능 반환의 일련의 과정을 한번에 진행할 수 있게 해준다.

# cross_val_predict()
#   cross_val_score과 비슷하지만, cross_val_score는 교차 검증 폴드에 대한 평균을 취하는 반면,
#   cross_val_predict는 구별되지 않은 개별 모델의 레이블을 반환한다.

"""
참조: https://velog.io/@aimaimee/sklearn.modelselection-crossvalidate-crossvalpredict-crossvalscore-%EA%B3%B5%EB%B6%80
"""
None

# 하이퍼파라미터 튜닝 GridSearchCV
교차검증과 최적의 하이퍼 파라미터 찾기

In [ ]:
# 용어 상기!

# ■ 모델파라미터: 머신러닝 모델이 학습하는 파라미터

# ■ 하이퍼파라미터: 모델이 학습할 수 없어서 '사용자'가 '학습전에 지정'해야만 하는 파라미터
#   => 사이킷런에서는 이런 하이퍼파라미터는 모두 클래스나 메소드의 매개변수로 표현된다

In [ ]:
# 최적의 하이퍼 파라미터를 찾기 위해 하이퍼파라미터 튜닝하는 작업은 어떻게 진행하나?
# step1 : 라이브러리가 제공하는 기본값을 그대로 사용해 모델 훈련.
# step2 : 검증세트의 점수나 교차 검증을 통해서 매개변수를 '조금씩' 바꿔본다.
#        모델마다 적게는 1~2개, 많게는 5~6개의 매개변수가 제공되는데...
#        이 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증 수행해야 한다.

In [ ]:
# ※ 참고] 사람의 개입없이 하이퍼 파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML' 이라고 부른다.

In [ ]:
# 위 방식의 문제점!
#  가령, DecisionTree 모델에서
#  일단 최적의 max_depth= 값을 찾았다고 치자.
#   그 max_depth 값을 '고정'하고
#  또 다른 하이퍼파라미터인 min_samples_split= 을 바꿔가면서 최적의 값을 찾았다.

# 이런식으로 최적값을 찾아가는게 맞을까?  -> NO!!!!

# max_depth= 의 최적값은 min_samples_split= 값이 바뀌면 '함께' 바뀐다.
# 즉, 이 두 매개변수를 '동시' 에 바꿔가며 최적의 값을 찾아내야만 한다!

# 그런데 매개변수가 많아지면 복잡해진다...

# => 이를 편리하게 수행해주는게 '그리드 서치 (GridSearch)' 다.


In [ ]:
# GridSearchCV
# cross validation 을 여러번 반복해서, 여기에서 최적의 하이퍼 파라미터 를 찾음

# GridSearchCV 는 '하이퍼 파라미터 탐색' + '교차검증' 을 한번에 수행

# ※ 이하에서 '파라미터' 라 함은 '하이퍼 파라미터' 를 말하는 겁니다.


In [34]:
from sklearn.model_selection import GridSearchCV

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

```python
class sklearn.model_selection.GridSearchCV(
  estimator,  # 탐색대상 모델
  param_grid, # 탐색 파라미터와 값들, dict
  *,
  scoring=None,
  n_jobs=None,  # 병렬실행에 사용할 CPU 코어수
  refit=True,
  cv=None,  # fold 개수, 5-fold 수행
  verbose=0,
  pre_dispatch='2*n_jobs',
  error_score=nan,
  return_train_score=False)
```  

## 한개의 파라미터 탐색

In [ ]:
# 간단한 예시를 작성해보자
# 기본 매개변수를 사용한 결정트리 모델에서 min_impurity_decrease= 의 최적값을 찾아보자!

In [35]:
# estimator(모델)가 갖고 있는 하이퍼 파라미터들 확인
DecisionTreeClassifier().get_params()


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [36]:
# '탐색할 매개변수'와 '탐색할 값(들)의 리스트' 를 dict로 만든다
#   min_impurity_decrease= 는 노드를 분할하기 위한 불순도 감소 최소량

min_impurity_decrease = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
params = {'min_impurity_decrease': min_impurity_decrease}


In [37]:
# GridSearchCV 클래스에 '탐색대상모델' 과 params 변수를 전달하여 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, cv=5)


In [ ]:
# 모델 훈련하듯이 fit() 호출

# cv=5 : 5-fold
#  min_impurity_decrease 값마다 5-fold 수행

# => 5-fold x 5 = 25개의 모델을 훈련한다!

# 병렬작업을 위해 n_jobs=-1 지정 (CPU 코어 최대 활용.)


In [38]:
# 참고] CPU 코어 개수 확인
import multiprocessing
multiprocessing.cpu_count()

# n_jobs=multiprocessing.cpu_count()  동일하다!

2

In [39]:
# 그리드서치 수행
gs.fit(train_input, train_target)

# ↑수행결과는 gs 객체에 담겨 있다. (리턴값 gs다 )

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [40]:

print("최적 점수 : {}".format(gs.best_score_))
print("최적 파라미터 : {}".format(gs.best_params_)   )
# ↑ 그리드 서치로 찾은 최적의 매개변수

최적 점수 : 0.8681929740134745
최적 파라미터 : {'min_impurity_decrease': 0.0001}


In [41]:
# 최적 의 파라미터로 학습된 모델
dt = gs.best_estimator_


In [42]:
dt.score(train_input, train_target)

0.9615162593804117

In [43]:
# 각 매개변수에서 수행한 교차 검증 결과 cv_results_ 속성에 담김
print(gs.cv_results_)


{'mean_fit_time': array([0.0105988 , 0.00907712, 0.00867763, 0.00806465, 0.00731797]), 'std_fit_time': array([0.00045929, 0.00021112, 0.00012564, 0.00022188, 0.00082451]), 'mean_score_time': array([0.0018292 , 0.00152583, 0.00142632, 0.00151753, 0.00146575]), 'std_score_time': array([3.98255137e-04, 8.46328062e-05, 3.82626092e-05, 1.47137066e-04,
       2.29425500e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]

In [44]:
# 이를 DataFrame 으로 변환하여 보자
pd.DataFrame(gs.cv_results_)

# 5 row : 5개의 min_impurity_decrease= 탐색값에 CV 수행

# 컬럼 split0_test_score ~ split4_test_score <- 5번의 CV 스코어
# 컬럼 mean_test_score  <- 5번의 CV 점수의 평균값 .  이중의 최댓값이 best_score_



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010599,0.000459,0.001829,0.000398,0.0001,{'min_impurity_decrease': 0.0001},0.869231,0.868269,0.882579,0.867180,0.853705,0.868193,0.009154,1
1,0.009077,0.000211,0.001526,0.000085,0.0002,{'min_impurity_decrease': 0.0002},0.871154,0.863462,0.876805,0.854668,0.856593,0.864536,0.008437,5
2,0.008678,0.000126,0.001426,0.000038,0.0003,{'min_impurity_decrease': 0.0003},0.869231,0.859615,0.875842,0.850818,0.869105,0.864922,0.008745,4
3,0.008065,0.000222,0.001518,0.000147,0.0004,{'min_impurity_decrease': 0.0004},0.869231,0.863462,0.881617,0.848893,0.875842,0.867809,0.011260,2
4,0.007318,0.000825,0.001466,0.000229,0.0005,{'min_impurity_decrease': 0.0005},0.865385,0.869231,0.882579,0.849856,0.871030,0.867616,0.010570,3


In [45]:
# 각 매개변수에서 수행한 교차 검증의 평균점수는 cv_results_ 속성
gs.cv_results_['mean_test_score']


array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [47]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(best_index)
print(gs.cv_results_['params'][best_index])   # gs.best_params_ 와 동일

0
{'min_impurity_decrease': 0.0001}


## Grid search 과정 정리
1. 탐색할 매개변수 및 탐색할 값들 지정
1. 훈련세트에서 그리드 서치 수행.
  - 결과: 최상의 평균점수가 나오는 매개변수 조합을 찾아낸다.
  - 이 조합은 그리드 서치 객체에 저장되어 있다
1. 그리드 서치는 최상의 매개변수에서 (교차검증에 사용한 훈련 세트가 아니라) 전체 훈련세트를 사용해 최종 모델을 훈련한다.  이 모델도 그리드 서치 객체에 저장된다.

## 복잡한 매개변수 조합 탐색

In [ ]:
# 이제 좀더 복잡한 매개변수 조합을 탐색해 봅시다.

# 결정 트리에서
#   min_impurity_decrease= 는 노드를 분할하기 위한 불순도 감소 최소량을 지정.
#   max_depth= 로 트리의 깊이를 제한
#   min_samples_split= 으로 노드를 나누기 위한 최소 샘플 수 지정


In [48]:
# 탐색할 하이퍼 파라미터(들)을 dict 로 준비


params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # 9개의 값
          'max_depth': range(5, 20, 1),  # 15개의 값
          'min_samples_split': range(2, 100, 10)  # 10개의 값
          }

params

{'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
        0.0009]),
 'max_depth': range(5, 20),
 'min_samples_split': range(2, 100, 10)}

In [ ]:
# 위 매개변수로 수행할 교차검증 횟수는? : 9 x 15 x 10 = 1,350개의 조합

# 기본 5-폴드교차 검증 수행하면 만들어지는 모델의 수는?: 1,350 x 5fold = 6,750개!

In [49]:
# GridSearch 수행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [50]:
# 최상의 매개변수 조합은?
gs.best_params_


{'max_depth': 14,
 'min_impurity_decrease': np.float64(0.0004),
 'min_samples_split': 12}

In [51]:
# 최상의 교차검증 점수도 확인해보자
print(gs.best_score_)
print(np.max(gs.cv_results_['mean_test_score']))


0.8683865773302731
0.8683865773302731


In [ ]:
# 만약, 매개변수의 탐색번위 보다 더 좁거나, 더 넓은 간격으로 시도해볼수도 있을까?


## 랜덤 서치 RandomizedSearchCV
Random Search

In [ ]:
# 매개변수 값이 '수치' 일때는 갑의 범위나 간격을 미리 정하기 어려울수도 있다.
# 또한 너무 많은 매개변수 조건은 그리드 서치 시간이 오래 걸리게 할수 있다.

# 이럴때 '랜덤 서치' 사용!
# 랜덤서치에는 값의 목록을 전달하는게 아니라, 매개변수 샘플링 하는 '확률 분포 객체' 를 전달

# ↓ scipy 에서 2개의 확률분포 클래스를 임포트 해봅시다

In [ ]:
# ※ 싸이파이(scipy) 는 파이썬의 기본 과학 라이브러리중 하나
#   https://scipy.org/
#   적분, 보간, 선형대수, 확률 등을 포함한 수치 계산 전용 라이브러리 입니다
#   사이킷런은 넘파이와 싸이파이 기능을 많이 사용합니다.  코랩에도 이미 설치 되어 있다.

In [52]:
from scipy.stats import uniform, randint

# uniform(), randint() 는 주어진 범위에서 고르게 램덤 값을 뽑습니다.
# => 균등분포 에서 랜덤 샘플링

# uniform() : 실수값 램덤 샘플링
# randint() : 정수값 램덤 샘플링

In [62]:
rgen = randint(0, 10)   # [0, 10)  사이 범위를 갖는 randint 객체 생성하고
print(type(rgen))
rgen.rvs(10)  # 10개의 숫자를 램덤 샘플링

<class 'scipy.stats._distn_infrastructure.rv_discrete_frozen'>


array([1, 3, 3, 0, 0, 7, 8, 4, 2, 7])

In [63]:
# ↑ 10개만 샘플링하면 고르게 샘플링 되는것 같지 않다.

# 샘플링 숫자를 늘리면 쉽게 확인할수 있다
# 1000개를 샘플링해서 각 숫자의 개수를 세어 보자

np.unique(rgen.rvs(1000), return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101,  94, 110,  98, 102,  94,  95, 103, 108,  95]))

In [64]:
# uniform() 의 사용 법도 동일하다

ugen = uniform(0, 1)   # [0.0, 1.0) 사이에서 실수를 램덤 샘플링
ugen.rvs(10)  # 10개


array([0.14383116, 0.72974659, 0.92061121, 0.53633432, 0.14745434,
       0.78744704, 0.53033031, 0.26731133, 0.26344048, 0.76800743])

In [69]:
# 탐색할 매개변수 준비
# 여기서는 min_samples_leaf=  를 탐색대상에 추가해 봅니다.
#      ↑ 이는 리프노드가 되기 위한 최소 샘플의 개수 입니다.
#       어떤 노드가 분할하여 만들어질 자식 노드의 샘플수가 이 값보다 작을 경우 분할하지 않습니다

                            # ↓ TODO
params = {'min_impurity_decrease': uniform(0.0001, 0.001),     # 0.0001 ~ 0.001 사이의 실숫값 샘플링
          'max_depth': randint(20, 50),         # 20~50 사이의 정수
          'min_samples_split': randint(2, 25), # 2 ~ 25 사이의 정수
          'min_samples_leaf': randint(1, 25),  # 1 ~ 25 사이의 정수
          }

# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV 의
# n_iter= 매개변수에서 지정

In [65]:
from sklearn.model_selection import RandomizedSearchCV

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

```python
class sklearn.model_selection.RandomizedSearchCV(
  estimator,
  param_distributions,  # 확률분포 객체를 가진 params
  *,
  n_iter=10, # 샘플링 횟수
  scoring=None,
  n_jobs=None,
  refit=True,
  cv=None,
  verbose=0,
  pre_dispatch='2*n_jobs', random_state=None,
  error_score=nan, r
  eturn_train_score=False)
```


In [70]:
gs = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    params,
    n_iter=100, # 샘플링 횟수
    n_jobs=-1,
    random_state=42,
    cv=5,
)


In [71]:
# 랜덤서치 교차검증 수행!
gs.fit(train_input, train_target)


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b2b6fa58ce0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b2b6fa589e0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b2b6fa5cce0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b2b6fbae600>},
                   random_state=42)

In [72]:
# 최적의 매개변수 조합 출력
gs.best_params_


{'max_depth': 39,
 'min_impurity_decrease': np.float64(0.00034102546602601173),
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [73]:
# 최적의 교차 검증 점수도 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [74]:
# 최적의 모델은 이미 전체 훈련세트 (train_input, train_target) 으로 훈련되어 있다
dt = gs.best_estimator_

# 위 모델을 최종 모델로 결정하고 테스트세트의 성능도 확인해보자
print(dt.score(test_input, test_target))


0.86


# 실습

# 확인문제

In [ ]:
# TODO